# Collaborative Filtering Using Singular Value Decomposition (SVD)

Importing Libraries.

In [3]:
import pandas as pd
import tarfile
from tqdm import tqdm
import json
import numpy as np
import time
from copy import deepcopy

Load all data.

In [4]:
data = []
with open('Digital_Music.json', 'r') as f:
    for l in tqdm(f):
        data.append(json.loads(l))

1584082it [00:29, 53984.45it/s]


Displaying headers.

In [5]:
# show data
print(data[0])

{'overall': 5.0, 'verified': True, 'reviewTime': '12 22, 2013', 'reviewerID': 'A1ZCPG3D3HGRSS', 'asin': '0001388703', 'style': {'Format:': ' Audio CD'}, 'reviewerName': 'mark l. massey', 'reviewText': 'This is a great cd full of worship favorites!!  All time great Keith green songs. His best album by far.', 'summary': 'Great worship cd', 'unixReviewTime': 1387670400}


Converting to data frame.

In [6]:
Rating = pd.DataFrame(data)
   

Viewing the data.

In [7]:
Rating.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5.0,True,"12 22, 2013",A1ZCPG3D3HGRSS,0001388703,{'Format:': ' Audio CD'},mark l. massey,This is a great cd full of worship favorites!!...,Great worship cd,1387670400,NaN,NaN
1,5.0,True,"09 11, 2013",AC2PL52NKPL29,0001388703,{'Format:': ' Audio CD'},Norma Mushen,"So creative! Love his music - the words, the ...",Gotta listen to this!,1378857600,NaN,NaN
2,5.0,True,"03 2, 2013",A1SUZXBDZSDQ3A,0001388703,{'Format:': ' Audio CD'},Herbert W. Shurley,"Keith Green, gone far to early in his carreer,...",Great approach still gets the message out,1362182400,NaN,NaN
3,5.0,True,"12 2, 2012",A3A0W7FZXM0IZW,0001388703,{'Format:': ' Audio CD'},Mary M Raybell,Keith Green had his special comedy style of Ch...,Great A must have,1354406400,NaN,NaN
4,5.0,False,"01 7, 2012",A12R54MKO17TW0,0001388703,{'Format:': ' Audio CD'},J. Bynum,Keith Green / So you wanna go back to Egypt......,A great one from Keith with a guest appearance...,1325894400,6,NaN


Load all metadata.

In [8]:

meta_data = []
with open('meta_Digital_Music.json', 'r') as f:
    for l in tqdm(f):
        meta_data.append(json.loads(l))

74347it [00:03, 19578.07it/s]


Displaying headers.

In [9]:
print(meta_data[0])

{'category': [], 'tech1': '', 'description': [], 'fit': '', 'title': 'Master Collection Volume One', 'also_buy': ['B000002UEN', 'B000008LD5', 'B01J804JKE', '7474034352', 'B004ZLBTXW', 'B000008LDH', 'B000TGKXJ8', 'B0000AM6KG', 'B0001XJ372', 'B001CFLHMC', 'B0007VO57G', 'B000005KVI', 'B0000251O2', 'B07FDMZ233', 'B000008LD9', 'B000008LDL', '0006935257', 'B07BHDLHH9', 'B00004WJ8I', 'B000008LDC', 'B000008LDI', 'B000002BLF', 'B000KNOC5G', 'B00079YLZ4', 'B000002GHQ', 'B07DXQ4SY4', 'B01NAIVYTN', '0785273425', 'B000E8CM5O', 'B000008N3O', 'B0000288NN', '0439763096', 'B000A79M1G', 'B00005OACU', 'B000008LDG', 'B00000JY96', 'B0002N61QC', 'B000008LDK', 'B00004UU0Y', 'B0000275JN', 'B000008LDN', 'B000008LDP', 'B000008ORJ'], 'image': [], 'tech2': '', 'brand': 'John Michael Talbot', 'feature': [], 'rank': '58,291 in CDs & Vinyl (', 'also_view': ['B000002UEN', 'B000008LD5', '7474034352', 'B000008LDH', 'B004ZLBTXW', 'B0001XJ372', 'B001CFLHMC', 'B01J804JKE', 'B000008LDC', 'B000TGKXJ8', '0819815802', 'B0007V

Converting to data frame.

In [10]:
meta = pd.DataFrame(meta_data)

Viewing the data.

In [11]:
meta.head()

,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,details
0,[],,[],,Master Collection Volume One,"[B000002UEN, B000008LD5, B01J804JKE, 747403435...",[],,John Michael Talbot,[],"58,291 in CDs & Vinyl (","[B000002UEN, B000008LD5, 7474034352, B000008LD...","<img src=""https://images-na.ssl-images-amazon....",,,$18.99,0001377647,NaN
1,[],,[],,Hymns Collection: Hymns 1 &amp; 2,"[5558154950, B00014K5V4]",[],,Second Chapter of Acts,[],"93,164 in CDs & Vinyl (","[B000008KJ3, B000008KJ0, 5558154950, B000UN8KZ...","<img src=""https://images-na.ssl-images-amazon....",,,,0001529145,NaN
2,[],,[],,Early Works - Don Francisco,"[B00004RC05, B003H8F4NA, B003ZFVHPO, B003JMP1Z...",[],,Don Francisco,[],"875,825 in CDs & Vinyl (","[B003H8F4NA, B003ZFVHPO, B003JMP1ZK, B00004RC0...","<img src=""https://images-na.ssl-images-amazon....",,,,0001527134,NaN
3,[],,[],,So You Wanna Go Back to Egypt,"[B0000275QQ, 0001393774, 0001388312, B0016CP2G...",[],,Keith Green,[],"203,263 in CDs & Vinyl (","[B00000I7JO, B0016CP2GS, 0001393774, B0000275Q...","<img src=""https://images-na.ssl-images-amazon....",,,$13.01,0001388703,NaN
4,[],,[1. Losing Game 2. I Can't Wait 3. Didn't He S...,,Early Works - Dallas Holm,"[B0002N4JP2, 0760131694, B00002EQ79, B00150K8J...",[],,Dallas Holm,[],"399,269 in CDs & Vinyl (","[B0002N4JP2, 0760131694, B00150K8JC, B003MTXNV...","<img src=""https://images-na.ssl-images-amazon....",,,,0001526146,NaN


Merging the two data frames.

In [12]:
df_merge_col = pd.merge(Rating, meta, on ='asin')

Viewing the merged data frame.

In [13]:
df_merge_col.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,...,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,details
0,5.0,True,"12 22, 2013",A1ZCPG3D3HGRSS,0001388703,{'Format:': ' Audio CD'},mark l. massey,This is a great cd full of worship favorites!!...,Great worship cd,1387670400,...,,Keith Green,[],"203,263 in CDs & Vinyl (","[B00000I7JO, B0016CP2GS, 0001393774, B0000275Q...","<img src=""https://images-na.ssl-images-amazon....",,,$13.01,NaN
1,5.0,True,"09 11, 2013",AC2PL52NKPL29,0001388703,{'Format:': ' Audio CD'},Norma Mushen,"So creative! Love his music - the words, the ...",Gotta listen to this!,1378857600,...,,Keith Green,[],"203,263 in CDs & Vinyl (","[B00000I7JO, B0016CP2GS, 0001393774, B0000275Q...","<img src=""https://images-na.ssl-images-amazon....",,,$13.01,NaN
2,5.0,True,"03 2, 2013",A1SUZXBDZSDQ3A,0001388703,{'Format:': ' Audio CD'},Herbert W. Shurley,"Keith Green, gone far to early in his carreer,...",Great approach still gets the message out,1362182400,...,,Keith Green,[],"203,263 in CDs & Vinyl (","[B00000I7JO, B0016CP2GS, 0001393774, B0000275Q...","<img src=""https://images-na.ssl-images-amazon....",,,$13.01,NaN
3,5.0,True,"12 2, 2012",A3A0W7FZXM0IZW,0001388703,{'Format:': ' Audio CD'},Mary M Raybell,Keith Green had his special comedy style of Ch...,Great A must have,1354406400,...,,Keith Green,[],"203,263 in CDs & Vinyl (","[B00000I7JO, B0016CP2GS, 0001393774, B0000275Q...","<img src=""https://images-na.ssl-images-amazon....",,,$13.01,NaN
4,5.0,False,"01 7, 2012",A12R54MKO17TW0,0001388703,{'Format:': ' Audio CD'},J. Bynum,Keith Green / So you wanna go back to Egypt......,A great one from Keith with a guest appearance...,1325894400,...,,Keith Green,[],"203,263 in CDs & Vinyl (","[B00000I7JO, B0016CP2GS, 0001393774, B0000275Q...","<img src=""https://images-na.ssl-images-amazon....",,,$13.01,NaN


Displaying the headers.

In [14]:
df_merge_col.columns

Index(['overall', 'verified', 'reviewTime', 'reviewerID', 'asin', 'style',
       'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'vote',
       'image_x', 'category', 'tech1', 'description', 'fit', 'title',
       'also_buy', 'image_y', 'tech2', 'brand', 'feature', 'rank', 'also_view',
       'main_cat', 'similar_item', 'date', 'price', 'details'],
      dtype='object')

 # Data pre-processing

Dropping least important columns.

In [15]:
rating = df_merge_col.drop(['style', 'category','tech1','fit','also_buy','tech2','similar_item','date','details','description','image_x','image_y','feature','also_view','main_cat'], axis=1)

Viewing the reduced dimension of the data frame.

In [16]:
rating.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,title,brand,rank,price
0,5.0,True,"12 22, 2013",A1ZCPG3D3HGRSS,0001388703,mark l. massey,This is a great cd full of worship favorites!!...,Great worship cd,1387670400,NaN,So You Wanna Go Back to Egypt,Keith Green,"203,263 in CDs & Vinyl (",$13.01
1,5.0,True,"09 11, 2013",AC2PL52NKPL29,0001388703,Norma Mushen,"So creative! Love his music - the words, the ...",Gotta listen to this!,1378857600,NaN,So You Wanna Go Back to Egypt,Keith Green,"203,263 in CDs & Vinyl (",$13.01
2,5.0,True,"03 2, 2013",A1SUZXBDZSDQ3A,0001388703,Herbert W. Shurley,"Keith Green, gone far to early in his carreer,...",Great approach still gets the message out,1362182400,NaN,So You Wanna Go Back to Egypt,Keith Green,"203,263 in CDs & Vinyl (",$13.01
3,5.0,True,"12 2, 2012",A3A0W7FZXM0IZW,0001388703,Mary M Raybell,Keith Green had his special comedy style of Ch...,Great A must have,1354406400,NaN,So You Wanna Go Back to Egypt,Keith Green,"203,263 in CDs & Vinyl (",$13.01
4,5.0,False,"01 7, 2012",A12R54MKO17TW0,0001388703,J. Bynum,Keith Green / So you wanna go back to Egypt......,A great one from Keith with a guest appearance...,1325894400,6,So You Wanna Go Back to Egypt,Keith Green,"203,263 in CDs & Vinyl (",$13.01


Replacing Nan values with zeros.

In [17]:
rating['vote'] = rating['vote'].fillna(0) 

Viewing the updated data frame.

In [18]:
rating.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,title,brand,rank,price
0,5.0,True,"12 22, 2013",A1ZCPG3D3HGRSS,0001388703,mark l. massey,This is a great cd full of worship favorites!!...,Great worship cd,1387670400,0,So You Wanna Go Back to Egypt,Keith Green,"203,263 in CDs & Vinyl (",$13.01
1,5.0,True,"09 11, 2013",AC2PL52NKPL29,0001388703,Norma Mushen,"So creative! Love his music - the words, the ...",Gotta listen to this!,1378857600,0,So You Wanna Go Back to Egypt,Keith Green,"203,263 in CDs & Vinyl (",$13.01
2,5.0,True,"03 2, 2013",A1SUZXBDZSDQ3A,0001388703,Herbert W. Shurley,"Keith Green, gone far to early in his carreer,...",Great approach still gets the message out,1362182400,0,So You Wanna Go Back to Egypt,Keith Green,"203,263 in CDs & Vinyl (",$13.01
3,5.0,True,"12 2, 2012",A3A0W7FZXM0IZW,0001388703,Mary M Raybell,Keith Green had his special comedy style of Ch...,Great A must have,1354406400,0,So You Wanna Go Back to Egypt,Keith Green,"203,263 in CDs & Vinyl (",$13.01
4,5.0,False,"01 7, 2012",A12R54MKO17TW0,0001388703,J. Bynum,Keith Green / So you wanna go back to Egypt......,A great one from Keith with a guest appearance...,1325894400,6,So You Wanna Go Back to Egypt,Keith Green,"203,263 in CDs & Vinyl (",$13.01


Hold out last review

In [19]:
ratings_reviewer_date = rating.loc[:, ['reviewerID','reviewTime']]

Viewing the rating and date. 

In [20]:
ratings_reviewer_date

,reviewerID,reviewTime
0,A1ZCPG3D3HGRSS,"12 22, 2013"
1,AC2PL52NKPL29,"09 11, 2013"
2,A1SUZXBDZSDQ3A,"03 2, 2013"
3,A3A0W7FZXM0IZW,"12 2, 2012"
4,A12R54MKO17TW0,"01 7, 2012"
...,...,...
182821,A11GYQ5Y9GHH0Z,"08 18, 2016"
182822,A21X4OZ5GLIJCM,"09 8, 2016"
182823,A1T4M8J6BKRPUI,"06 23, 2017"
182824,A2746S1ODNDAVH,"01 24, 2017"


Indexing the date.

In [21]:
ratings_reviewer_date.reviewTime = pd.to_datetime(ratings_reviewer_date.reviewTime)

Viewing the indexed date.

In [22]:
ratings_reviewer_date

,reviewerID,reviewTime
0,A1ZCPG3D3HGRSS,2013-12-22
1,AC2PL52NKPL29,2013-09-11
2,A1SUZXBDZSDQ3A,2013-03-02
3,A3A0W7FZXM0IZW,2012-12-02
4,A12R54MKO17TW0,2012-01-07
...,...,...
182821,A11GYQ5Y9GHH0Z,2016-08-18
182822,A21X4OZ5GLIJCM,2016-09-08
182823,A1T4M8J6BKRPUI,2017-06-23
182824,A2746S1ODNDAVH,2017-01-24


Creating an index for the rating data.

In [23]:
index_hold = ratings_reviewer_date.groupby(['reviewerID'], sort=False)['reviewTime'].transform(max) == ratings_reviewer_date['reviewTime']

In [47]:
ratings_holdout = rating[index_hold]

In [48]:
ratings_traincv_ = rating[~index_hold]

In [49]:
ratings_reviewer_date = ratings_traincv_.loc[:, ['reviewerID','reviewTime']]

In [50]:
ratings_reviewer_date

,reviewerID,reviewTime
4,A12R54MKO17TW0,"01 7, 2012"
11,A1DOF5GHOWGMW6,"11 4, 2014"
16,A27P44I54RUMDC,"04 15, 2014"
31,A3BQ84G90BRVSG,"06 25, 2016"
48,A27P44I54RUMDC,"07 18, 2014"
...,...,...
182792,A3W4SX5CA1MK2I,"04 22, 2017"
182794,AANTLC5FKVMLJ,"12 1, 2016"
182802,A1ANRKHR4QWXJU,"10 28, 2016"
182806,A1F30W5APHR6F0,"08 6, 2016"


In [51]:
index_holdout = ratings_reviewer_date.groupby(['reviewerID'], sort=False)['reviewTime'].transform(max) == ratings_reviewer_date['reviewTime']

In [52]:
ratings_cv_ = ratings_traincv_[index_holdout]

In [53]:
ratings_train_ = ratings_traincv_[~index_holdout]

Importing the surprise library.

In [54]:
from surprise import SVD
from surprise import accuracy
from surprise import Reader
from surprise import Dataset
from surprise import BaselineOnly

Using the reader function to scale the ratings.

In [55]:
reader = Reader(rating_scale = (0.0, 5.0))

The minimum rating possible is 1.

In [56]:
min(rating['overall'])

1.0

The maximum rating possible is 5.

In [57]:
max(rating['overall'])

5.0

In [58]:
ratings_val = ratings_holdout.loc[:,['reviewerID','title','overall']] 

Training Data

In [59]:
trainset = ratings_train_.loc[:,['reviewerID','title','overall']] 
trainset.columns = ['reviewerID','title','overall']

Validating Data

In [60]:
valset = ratings_val.loc[:, ['reviewerID','title','overall']]
valset.columns = ['reviewerID','title','overall']

Testing Data

In [62]:
testset = ratings_holdout.loc[:, ['reviewerID','title','overall']]
testset.columns = ['reviewerID','title','overall']

In [63]:
reader = Reader(rating_scale = (0.0, 5.0))
train_data = Dataset.load_from_df(trainset[['reviewerID','title','overall']], reader)
val_data = Dataset.load_from_df(valset[['reviewerID','title','overall']], reader)
test_data = Dataset.load_from_df(testset[['reviewerID','title','overall']], reader)

Training the model.

In [64]:
train_sr = train_data.build_full_trainset()
val_sr_before = val_data.build_full_trainset()
val_sr = val_sr_before.build_testset()
test_sr_before = test_data.build_full_trainset()
test_sr = test_sr_before.build_testset()

Fitting the model

In [65]:
 algo = SVD()
 algo.fit(train_sr)           

Predictions or suggestion.

In [66]:
predictions = algo.test(val_sr)           

In [67]:
predictions

[Prediction(uid='A1ZCPG3D3HGRSS', iid='So You Wanna Go Back to Egypt', r_ui=5.0, est=4.531113904817443, details={'was_impossible': False}),
 Prediction(uid='AC2PL52NKPL29', iid='So You Wanna Go Back to Egypt', r_ui=5.0, est=4.531113904817443, details={'was_impossible': False}),
 Prediction(uid='A1SUZXBDZSDQ3A', iid='So You Wanna Go Back to Egypt', r_ui=5.0, est=4.531113904817443, details={'was_impossible': False}),
 Prediction(uid='A3A0W7FZXM0IZW', iid='So You Wanna Go Back to Egypt', r_ui=5.0, est=4.531113904817443, details={'was_impossible': False}),
 Prediction(uid='A25ZT87OMIPLNX', iid='So You Wanna Go Back to Egypt', r_ui=5.0, est=4.531113904817443, details={'was_impossible': False}),
 Prediction(uid='A3NVGWKHLULDHR', iid='So You Wanna Go Back to Egypt', r_ui=1.0, est=4.531113904817443, details={'was_impossible': False}),
 Prediction(uid='AT7OB43GHKIUA', iid='So You Wanna Go Back to Egypt', r_ui=5.0, est=4.531113904817443, details={'was_impossible': False}),
 Prediction(uid='A1H3X

The root square mean error score.

In [68]:
RMSE_tune = accuracy.rmse(predictions)

RMSE: 1.0074


Train and test on the optimal parameter.

In [70]:
start_time = time.time()
algo_real = SVD(n_epochs = 30, lr_all = 0.005, reg_all = 0.1)
algo_real.fit(train_sr)
opt_predictions = algo_real.test(test_sr)

In [71]:
opt_predictions 

[Prediction(uid='A1ZCPG3D3HGRSS', iid='So You Wanna Go Back to Egypt', r_ui=5.0, est=4.53508085399017, details={'was_impossible': False}),
 Prediction(uid='AC2PL52NKPL29', iid='So You Wanna Go Back to Egypt', r_ui=5.0, est=4.53508085399017, details={'was_impossible': False}),
 Prediction(uid='A1SUZXBDZSDQ3A', iid='So You Wanna Go Back to Egypt', r_ui=5.0, est=4.53508085399017, details={'was_impossible': False}),
 Prediction(uid='A3A0W7FZXM0IZW', iid='So You Wanna Go Back to Egypt', r_ui=5.0, est=4.53508085399017, details={'was_impossible': False}),
 Prediction(uid='A25ZT87OMIPLNX', iid='So You Wanna Go Back to Egypt', r_ui=5.0, est=4.53508085399017, details={'was_impossible': False}),
 Prediction(uid='A3NVGWKHLULDHR', iid='So You Wanna Go Back to Egypt', r_ui=1.0, est=4.53508085399017, details={'was_impossible': False}),
 Prediction(uid='AT7OB43GHKIUA', iid='So You Wanna Go Back to Egypt', r_ui=5.0, est=4.53508085399017, details={'was_impossible': False}),
 Prediction(uid='A1H3X1TW6Y7H